In [44]:
from dotenv import dotenv_values
import snowflake.connector
import pandas as pd
import numpy as np
from IPython.display import display, Markdown, Latex


In [ ]:
# there is probably a better way to do this. But this works for now
data_tables = ['AD','ELITE', 'GENIE', 'HTAN', 'NF', 'PSYCHENCODE']
table_names = ['SAGE.PORTAL_RAW.' + d for d in data_tables]
table_names

In [ ]:
config = dotenv_values(".env")
conn = snowflake.connector.connect(
    user=config['USER'],
    account=config['ACCOUNT_IDENTIFIER'],
    authenticator="externalbrowser", # FOR browser-based SSO for authentication since account uses Google account for login. It is organization-username
    warehouse=config['WAREHOUSE'],
    database=config['DATABASE'],
    role=config['ROLE'],
    login_timeout = 60,
    network_timeout=30,
    socket_timeout=10
)

In [140]:
# create cursor
cur = conn.cursor()

# Try to join all the tables together

sf_tables = {}

for t in table_names: 
    query = f"""
        SELECT * FROM {t}
    """

    cur.execute(query)

    # Retrieve results
    df = pd.concat([d for d in cur.fetch_pandas_batches()])
    df = df.reset_index(drop=True)
    df['TABLE'] = t
    sf_tables[t] = df

cur.close()

True

In [142]:
comb_df = pd.concat(sf_tables.values()).reset_index(drop=True)

original_shape = comb_df.shape
original_cols = sorted(comb_df.columns)
comb_df = comb_df.dropna(how='all', axis =1)
# removing empty lists and changing all nonetypes to nans
comb_df = comb_df.replace('[]', np.nan).fillna(value=np.nan)
# cleanup lists and values for new lines, double spaces and quotes
comb_df = comb_df.apply(lambda x: x.str.replace('\n|\s+|"', '', regex = True), axis = 1)
# drop empty columns
comb_df = comb_df.dropna(how='all', axis =1)
comb_df = comb_df[sorted(comb_df.columns)]

display(Markdown(f"""
|ORIGINAL|NEW|
|---|---|
|{original_shape}|{comb_df.shape}|
"""))


|ORIGINAL|NEW|
|---|---|
|(522977, 105)|(522977, 83)|


In [143]:
comb_df

,ACCESSTYPE,AGE,ALIGNMENTMETHOD,ANALYSISTYPE,ASSAY,ASSAYTARGET,BENEFACTORID,BRODMANNAREA,CELLTYPE,CHROMOSOME,...,STUDYID,STUDYNAME,TABLE,TERMINALDIFFERENTIATIONPOINT,TISSUE,TRANSPLANTATIONTYPE,TREATMENTTYPE,TUMORTYPE,TYPE,VERSION
0,NaN,NaN,NaN,genotypeimputation,[snpArray],NaN,syn5550382,NaN,NaN,NaN,...,NaN,NaN,SAGE.PORTAL_RAW.AD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,genotypeimputation,[snpArray],NaN,syn5550382,NaN,NaN,NaN,...,NaN,NaN,SAGE.PORTAL_RAW.AD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,genotypeimputation,[snpArray],NaN,syn5550382,NaN,NaN,NaN,...,NaN,NaN,SAGE.PORTAL_RAW.AD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,genotypeimputation,[snpArray],NaN,syn5550382,NaN,NaN,NaN,...,NaN,NaN,SAGE.PORTAL_RAW.AD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,genotypeimputation,[snpArray],NaN,syn5550382,NaN,NaN,NaN,...,NaN,NaN,SAGE.PORTAL_RAW.AD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522972,NaN,NaN,NaN,NaN,NaN,NaN,syn20729790,NaN,NaN,NaN,...,NaN,NaN,SAGE.PORTAL_RAW.PSYCHENCODE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522973,NaN,NaN,NaN,NaN,NaN,NaN,syn20729790,NaN,NaN,NaN,...,NaN,NaN,SAGE.PORTAL_RAW.PSYCHENCODE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522974,NaN,NaN,NaN,NaN,NaN,NaN,syn20729790,NaN,NaN,NaN,...,NaN,NaN,SAGE.PORTAL_RAW.PSYCHENCODE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522975,NaN,NaN,NaN,NaN,ATACSeq,NaN,syn5584622,NaN,NaN,NaN,...,NaN,NaN,SAGE.PORTAL_RAW.PSYCHENCODE,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
# pull out only columns found in all tables
# cols = {}

# for k,v in sf_tables.items(): 
#     cols[k] = set(v.columns)

# u = set.intersection(*cols.values())
# u

In [147]:
cols

{'SAGE.PORTAL_RAW.AD': {'ANALYSISTYPE',
  'ASSAY',
  'ASSAYTARGET',
  'BENEFACTORID',
  'CELLTYPE',
  'CHROMOSOME',
  'CONSORTIUM',
  'CREATEDBY',
  'CREATEDON',
  'CURRENTVERSION',
  'DATAFILEHANDLEID',
  'DATAFILESIZEBYTES',
  'DATASUBTYPE',
  'DATATYPE',
  'FILEFORMAT',
  'GRANTS',
  'GROUPS',
  'ID',
  'INDIVIDUALID',
  'INDIVIDUALIDSOURCE',
  'ISCONSORTIUMANALYSIS',
  'ISMODELSYSTEM',
  'ISMULTISPECIMEN',
  'LIBRARYPREP',
  'METABOLITETYPE',
  'METADATATYPE',
  'MODELSYSTEMNAME',
  'MODELSYSTEMTYPE',
  'MODIFIEDBY',
  'MODIFIEDON',
  'NAME',
  'NUCLEICACIDSOURCE',
  'ORGAN',
  'PARENTID',
  'PROJECTID',
  'RESOURCETYPE',
  'SEX',
  'SPECIES',
  'SPECIMENID',
  'SPECIMENIDSOURCE',
  'STUDY',
  'TABLE',
  'TISSUE',
  'TREATMENTTYPE'},
 'SAGE.PORTAL_RAW.ELITE': {'ANALYSISTYPE',
  'ASSAY',
  'CONSENT',
  'CONSORTIUM',
  'CURRENTVERSION',
  'DATASUBTYPE',
  'DATATYPE',
  'ETAG',
  'FILEFORMAT',
  'GRANTS',
  'ID',
  'ISCONSORTIUMANALYSIS',
  'ISMODELSYSTEM',
  'ISMULTISPECIMEN',
  'LIB

{'TABLE'}